In [1]:
import pandas as pd

In [2]:
df=pd.read_csv(r"ipd_axelrod_nn_dataset_mem7_random.csv")

In [3]:
df['payoff_difference']=df['A_cum_payoff']-df['B_cum_payoff']
df.drop(columns=['A_cum_payoff','B_cum_payoff'],inplace=True)

In [4]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

MEMORY = 7
BATCH_SIZE = 64
EPOCHS = 15
LR = 0.001
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

X_seq, y = [], []

for _, row in df.iterrows():
    seq = []
    for i in range(MEMORY, 0, -1):
        seq.append([row[f"B_last_{i}"]])   # ONLY opponent history

    X_seq.append(seq)
    y.append(row["A_next_move"])           # EXISTING COLUMN

X = np.array(X_seq, dtype=np.float32)      # (samples, 7, 1)
y = np.array(y, dtype=np.float32)

print("X shape:", X.shape)

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

class IPDDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X)
        self.y = torch.tensor(y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_loader = DataLoader(
    IPDDataset(X_train, y_train),
    batch_size=BATCH_SIZE,
    shuffle=True
)

test_loader = DataLoader(
    IPDDataset(X_test, y_test),
    batch_size=BATCH_SIZE,
    shuffle=False
)

class LSTMModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=1,
            hidden_size=32,
            batch_first=True
        )
        self.fc = nn.Linear(32, 1)

    def forward(self, x):
        _, (h, _) = self.lstm(x)
        return self.fc(h[-1]).squeeze(1)

model = LSTMModel().to(DEVICE)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for xb, yb in train_loader:
        xb = xb.to(DEVICE)
        yb = yb.to(DEVICE)

        optimizer.zero_grad()
        logits = model(xb)
        loss = criterion(logits, yb)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{EPOCHS} | Loss: {total_loss/len(train_loader):.4f}")


model.eval()
preds, true = [], []

with torch.no_grad():
    for xb, yb in test_loader:
        xb = xb.to(DEVICE)
        logits = model(xb)
        probs = torch.sigmoid(logits)
        preds.extend((probs > 0.5).cpu().numpy())
        true.extend(yb.numpy())

print("\nAccuracy:", accuracy_score(true, preds))
print("\nClassification Report:\n", classification_report(true, preds))

X shape: (2464, 7, 1)
Epoch 1/15 | Loss: 0.6832
Epoch 2/15 | Loss: 0.6416
Epoch 3/15 | Loss: 0.5259
Epoch 4/15 | Loss: 0.4919
Epoch 5/15 | Loss: 0.4841
Epoch 6/15 | Loss: 0.4800
Epoch 7/15 | Loss: 0.4758
Epoch 8/15 | Loss: 0.4725
Epoch 9/15 | Loss: 0.4734
Epoch 10/15 | Loss: 0.4688
Epoch 11/15 | Loss: 0.4678
Epoch 12/15 | Loss: 0.4694
Epoch 13/15 | Loss: 0.4688
Epoch 14/15 | Loss: 0.4694
Epoch 15/15 | Loss: 0.4643

Accuracy: 0.7565922920892495

Classification Report:
               precision    recall  f1-score   support

         0.0       0.69      0.85      0.76       227
         1.0       0.84      0.68      0.75       266

    accuracy                           0.76       493
   macro avg       0.77      0.76      0.76       493
weighted avg       0.77      0.76      0.76       493



In [6]:
def predict_next_move(model, b_last):
    """
    b_last: list of 7 values -> B_last_7 to B_last_1
    """
    model.eval()

    x = np.array(b_last, dtype=np.float32).reshape(1, MEMORY, 1)
    x = torch.tensor(x).to(DEVICE)

    with torch.no_grad():
        logits = model(x)
        prob = torch.sigmoid(logits).item()
        pred = 1 if prob > 0.5 else 0

    return pred, prob


print("\nEnter B's last 7 moves (B_last_7 → B_last_1)")
b_moves = [int(input(f"B_last_{i}: ")) for i in range(7, 0, -1)]
pred, prob = predict_next_move(model, b_moves)

print("\nPredicted A_next_move:", "Cooperate (1)" if pred == 1 else "Defect (0)")


Enter B's last 7 moves (B_last_7 → B_last_1)
B_last_7: 1
B_last_6: 1
B_last_5: 1
B_last_4: 0
B_last_3: 0
B_last_2: 0
B_last_1: 1

Predicted A_next_move: Defect (0)
